In [1]:
# -*- coding: utf-8 -*-
"""
Protein regression (3 models)
- Outer CV: 5-fold GroupKFold (by sample_id)
- Inner CV: 3-fold GroupKFold hyperparameter search (manual grid)
- Models:
    1) PLSR  : patch-level training -> sample-level (median) evaluation
    2) SVR   : sample-level training/evaluation using sample-representative spectrum (median across patches)
    3) RF    : patch-level training -> sample-level (median) evaluation
- Preprocess:
    - SNV always
    - StandardScaler for PLSR & SVR
    - RF no scaler
- Outputs:
  - cv_model_comparison.csv
  - cv_fold_metrics_3models.csv
  - cv_oof_samples_3models.csv
  - best_model_oof_samples.csv
  - plots for best model only (parity / residuals)
"""

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import r2_score

from sklearn.cross_decomposition import PLSRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# ---------------- Basic utils ----------------
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def snv(X):
    """Standard Normal Variate (row-wise)."""
    X = np.asarray(X, dtype=float)
    mu = X.mean(axis=1, keepdims=True)
    sd = X.std(axis=1, keepdims=True)
    sd[sd == 0] = 1.0
    return (X - mu) / sd

def find_protein_col(columns):
    pats = [
        r'^protein(\s*content)?\s*(\(\s*%\s*\)|%)?$',
        r'^crude\s*protein(\s*content)?\s*(\(\s*%\s*\)|%)?$'
    ]
    cols = [str(c).strip() for c in columns]
    for c in cols:
        cl = c.lower()
        for p in pats:
            if re.match(p, cl):
                return c
    for c in cols:
        if 'protein' in c.lower():
            return c
    raise KeyError("Cannot find protein column. Please check column names.")

def sort_spec_cols(spec_cols):
    def _w2f(c):
        try:
            return float(str(c).replace("nm",""))
        except:
            return np.inf
    return sorted(spec_cols, key=_w2f)

def sample_level_aggregate_from_patch_preds(y_true_patch, y_pred_patch, sample_ids_patch, agg="median"):
    df = pd.DataFrame({
        "sample_id": np.asarray(sample_ids_patch).astype(str),
        "y_true":    np.asarray(y_true_patch, dtype=float),
        "y_pred":    np.asarray(y_pred_patch, dtype=float),
    })
    if agg == "median":
        agg_df = df.groupby("sample_id")[["y_true","y_pred"]].median()
    elif agg == "mean":
        agg_df = df.groupby("sample_id")[["y_true","y_pred"]].mean()
    else:
        raise ValueError("agg must be 'median' or 'mean'.")
    return agg_df.reset_index()

def sample_level_metrics_from_patch_preds(y_true_patch, y_pred_patch, sample_ids_patch, agg="median"):
    agg_df = sample_level_aggregate_from_patch_preds(y_true_patch, y_pred_patch, sample_ids_patch, agg=agg)
    r2  = r2_score(agg_df["y_true"], agg_df["y_pred"])
    e   = rmse(agg_df["y_true"], agg_df["y_pred"])
    return r2, e, agg_df

def sample_level_spectra(X_patch, y_patch, sample_ids, agg="median"):
    """
    Aggregate patch spectra to sample-representative spectra for SVR.
    Returns: X_s, y_s, sample_ids_unique
    """
    sid = np.asarray(sample_ids).astype(str)
    y_df = pd.DataFrame({"sample_id": sid, "y": np.asarray(y_patch, dtype=float)})
    y_s = y_df.groupby("sample_id")["y"].median()

    X_df = pd.DataFrame(np.asarray(X_patch, dtype=float))
    X_df["sample_id"] = sid
    if agg == "median":
        X_s = X_df.groupby("sample_id").median(numeric_only=True)
    elif agg == "mean":
        X_s = X_df.groupby("sample_id").mean(numeric_only=True)
    else:
        raise ValueError("agg must be 'median' or 'mean'.")

    X_s = X_s.loc[y_s.index]
    return X_s.to_numpy(dtype=float), y_s.to_numpy(dtype=float), y_s.index.to_numpy()

# ---------------- Path config ----------------
BASE_DIR = Path(__file__).resolve().parent

DATA_BASE = BASE_DIR / "Pea" / "flour"
PATCH_CSV = DATA_BASE / "pea_patch_dataset.csv"
OUT_DIR   = DATA_BASE / "plots_protein_reg_3models"

OUT_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_STATE = 42
AGG_METHOD = "median"

# ---------------- Load data ----------------
df = pd.read_csv(PATCH_CSV)

if "sample_id" not in df.columns:
    raise KeyError("Your patch CSV must contain a 'sample_id' column for GroupKFold.")

spec_cols = sort_spec_cols([c for c in df.columns if str(c).strip().endswith("nm")])
protein_col = find_protein_col(df.columns)

df = df.loc[df[protein_col].notna()].copy()

X_patch = df[spec_cols].to_numpy(dtype=float)
y_patch = df[protein_col].astype(float).to_numpy()
g_patch = df["sample_id"].astype(str).to_numpy()

print(f"[INFO] patches={len(df)} | samples={df['sample_id'].nunique()} | bands={len(spec_cols)}")
print(f"[INFO] protein column = {protein_col}")

# Sample-level spectra (SVR only)
X_s, y_s, g_s = sample_level_spectra(X_patch, y_patch, g_patch, agg=AGG_METHOD)
print(f"[INFO] SVR uses sample-level data: X={X_s.shape}, y={y_s.shape}")

# ---------------- CV ----------------
outer_cv = GroupKFold(n_splits=5)
inner_cv = GroupKFold(n_splits=3)

# ---------------- Model configs ----------------
def plsr_pipeline(n_components):
    return Pipeline([
        ("snv", FunctionTransformer(snv, validate=False)),
        ("scaler", StandardScaler()),
        ("model", PLSRegression(n_components=n_components))
    ])

def svr_pipeline(C, gamma, epsilon):
    return Pipeline([
        ("snv", FunctionTransformer(snv, validate=False)),
        ("scaler", StandardScaler()),
        ("model", SVR(kernel="rbf", C=C, gamma=gamma, epsilon=epsilon))
    ])

def rf_pipeline(n_estimators, max_depth, max_features, min_samples_leaf):
    return Pipeline([
        ("snv", FunctionTransformer(snv, validate=False)),
        ("model", RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_features=max_features,
            min_samples_leaf=min_samples_leaf,
            random_state=RANDOM_STATE,
            n_jobs=-1
        ))
    ])

# Compact but defensible grids
def plsr_grid(Xtr, gtr):
    max_k = int(min(20, Xtr.shape[1], len(np.unique(gtr)) - 1, Xtr.shape[0] - 1))
    return [{"n_components": k} for k in range(1, max_k + 1)]

SVR_GRID = [
    {"C": 10,  "gamma": "scale", "epsilon": 0.05},
    {"C": 100, "gamma": "scale", "epsilon": 0.05},
    {"C": 10,  "gamma": 0.01,    "epsilon": 0.05},
    {"C": 100, "gamma": 0.01,    "epsilon": 0.05},
]

RF_GRID = [
    {"n_estimators": n, "max_depth": d, "max_features": mf, "min_samples_leaf": msl}
    for n in [400, 800]
    for d in [None, 20, 40]
    for mf in ["sqrt", 0.3]
    for msl in [1, 2]
]

# ---------------- Nested CV runners ----------------
def run_plsr_nested():
    model_name = "PLSR"
    fold_rows, oof_rows = [], []

    fold_id = 1
    for tr, te in outer_cv.split(X_patch, y_patch, groups=g_patch):
        Xtr, Xte = X_patch[tr], X_patch[te]
        ytr, yte = y_patch[tr], y_patch[te]
        gtr, gte = g_patch[tr], g_patch[te]

        best_score, best_p = -np.inf, None
        for p in plsr_grid(Xtr, gtr):
            scores = []
            for itr, iva in inner_cv.split(Xtr, ytr, groups=gtr):
                mdl = plsr_pipeline(p["n_components"])
                mdl.fit(Xtr[itr], ytr[itr])
                pred_va = mdl.predict(Xtr[iva]).ravel()
                r2_va, _, _ = sample_level_metrics_from_patch_preds(ytr[iva], pred_va, gtr[iva], agg=AGG_METHOD)
                scores.append(r2_va)
            m = float(np.mean(scores))
            if m > best_score:
                best_score, best_p = m, p

        final = plsr_pipeline(best_p["n_components"])
        final.fit(Xtr, ytr)
        pred_te = final.predict(Xte).ravel()

        r2_s, rmse_s, agg_df = sample_level_metrics_from_patch_preds(yte, pred_te, gte, agg=AGG_METHOD)
        agg_df["fold"] = fold_id
        agg_df["model"] = model_name
        oof_rows.append(agg_df)

        fold_rows.append({
            "model": model_name,
            "fold": fold_id,
            "sample_R2": r2_s,
            "sample_RMSE": rmse_s,
            "best_params": str(best_p),
            "inner_best_R2_mean": best_score
        })

        print(f"[{model_name} | Fold {fold_id}] R2={r2_s:.4f} RMSE={rmse_s:.4f} | best={best_p}")
        fold_id += 1

    fold_df = pd.DataFrame(fold_rows)
    oof_df = pd.concat(oof_rows, ignore_index=True)
    r2_overall = r2_score(oof_df["y_true"], oof_df["y_pred"])
    rmse_overall = rmse(oof_df["y_true"], oof_df["y_pred"])

    summary = {
        "model": model_name,
        "R2_mean": float(fold_df["sample_R2"].mean()),
        "R2_std":  float(fold_df["sample_R2"].std(ddof=0)),
        "RMSE_mean": float(fold_df["sample_RMSE"].mean()),
        "RMSE_std":  float(fold_df["sample_RMSE"].std(ddof=0)),
        "R2_overall_oof": float(r2_overall),
        "RMSE_overall_oof": float(rmse_overall),
    }
    return summary, fold_df, oof_df

def run_rf_nested():
    model_name = "RF"
    fold_rows, oof_rows = [], []

    fold_id = 1
    for tr, te in outer_cv.split(X_patch, y_patch, groups=g_patch):
        Xtr, Xte = X_patch[tr], X_patch[te]
        ytr, yte = y_patch[tr], y_patch[te]
        gtr, gte = g_patch[tr], g_patch[te]

        best_score, best_p = -np.inf, None
        for p in RF_GRID:
            scores = []
            for itr, iva in inner_cv.split(Xtr, ytr, groups=gtr):
                mdl = rf_pipeline(p["n_estimators"], p["max_depth"], p["max_features"], p["min_samples_leaf"])
                mdl.fit(Xtr[itr], ytr[itr])
                pred_va = mdl.predict(Xtr[iva]).ravel()
                r2_va, _, _ = sample_level_metrics_from_patch_preds(ytr[iva], pred_va, gtr[iva], agg=AGG_METHOD)
                scores.append(r2_va)
            m = float(np.mean(scores))
            if m > best_score:
                best_score, best_p = m, p

        final = rf_pipeline(best_p["n_estimators"], best_p["max_depth"], best_p["max_features"], best_p["min_samples_leaf"])
        final.fit(Xtr, ytr)
        pred_te = final.predict(Xte).ravel()

        r2_s, rmse_s, agg_df = sample_level_metrics_from_patch_preds(yte, pred_te, gte, agg=AGG_METHOD)
        agg_df["fold"] = fold_id
        agg_df["model"] = model_name
        oof_rows.append(agg_df)

        fold_rows.append({
            "model": model_name,
            "fold": fold_id,
            "sample_R2": r2_s,
            "sample_RMSE": rmse_s,
            "best_params": str(best_p),
            "inner_best_R2_mean": best_score
        })

        print(f"[{model_name} | Fold {fold_id}] R2={r2_s:.4f} RMSE={rmse_s:.4f} | best={best_p}")
        fold_id += 1

    fold_df = pd.DataFrame(fold_rows)
    oof_df = pd.concat(oof_rows, ignore_index=True)
    r2_overall = r2_score(oof_df["y_true"], oof_df["y_pred"])
    rmse_overall = rmse(oof_df["y_true"], oof_df["y_pred"])

    summary = {
        "model": model_name,
        "R2_mean": float(fold_df["sample_R2"].mean()),
        "R2_std":  float(fold_df["sample_R2"].std(ddof=0)),
        "RMSE_mean": float(fold_df["sample_RMSE"].mean()),
        "RMSE_std":  float(fold_df["sample_RMSE"].std(ddof=0)),
        "R2_overall_oof": float(r2_overall),
        "RMSE_overall_oof": float(rmse_overall),
    }
    return summary, fold_df, oof_df

def run_svr_nested_samplelevel():
    """
    SVR is trained and evaluated at sample level (aggregated spectra),
    to avoid O(n^2) kernel scaling with 15000 patches.
    """
    model_name = "SVR_RBF"
    fold_rows, oof_rows = [], []

    fold_id = 1
    for tr, te in outer_cv.split(X_s, y_s, groups=g_s):
        Xtr, Xte = X_s[tr], X_s[te]
        ytr, yte = y_s[tr], y_s[te]
        gtr, gte = g_s[tr], g_s[te]

        best_score, best_p = -np.inf, None
        for p in SVR_GRID:
            scores = []
            for itr, iva in inner_cv.split(Xtr, ytr, groups=gtr):
                mdl = svr_pipeline(p["C"], p["gamma"], p["epsilon"])
                mdl.fit(Xtr[itr], ytr[itr])
                pred_va = mdl.predict(Xtr[iva]).ravel()
                scores.append(r2_score(ytr[iva], pred_va))
            m = float(np.mean(scores))
            if m > best_score:
                best_score, best_p = m, p

        final = svr_pipeline(best_p["C"], best_p["gamma"], best_p["epsilon"])
        final.fit(Xtr, ytr)
        pred_te = final.predict(Xte).ravel()

        r2_s_fold = r2_score(yte, pred_te)
        rmse_s_fold = rmse(yte, pred_te)

        # build oof table consistent columns
        agg_df = pd.DataFrame({
            "sample_id": gte.astype(str),
            "y_true": yte.astype(float),
            "y_pred": pred_te.astype(float),
        })
        agg_df["fold"] = fold_id
        agg_df["model"] = model_name
        oof_rows.append(agg_df)

        fold_rows.append({
            "model": model_name,
            "fold": fold_id,
            "sample_R2": r2_s_fold,
            "sample_RMSE": rmse_s_fold,
            "best_params": str(best_p),
            "inner_best_R2_mean": best_score
        })

        print(f"[{model_name} | Fold {fold_id}] R2={r2_s_fold:.4f} RMSE={rmse_s_fold:.4f} | best={best_p}")
        fold_id += 1

    fold_df = pd.DataFrame(fold_rows)
    oof_df = pd.concat(oof_rows, ignore_index=True)

    r2_overall = r2_score(oof_df["y_true"], oof_df["y_pred"])
    rmse_overall = rmse(oof_df["y_true"], oof_df["y_pred"])

    summary = {
        "model": model_name,
        "R2_mean": float(fold_df["sample_R2"].mean()),
        "R2_std":  float(fold_df["sample_R2"].std(ddof=0)),
        "RMSE_mean": float(fold_df["sample_RMSE"].mean()),
        "RMSE_std":  float(fold_df["sample_RMSE"].std(ddof=0)),
        "R2_overall_oof": float(r2_overall),
        "RMSE_overall_oof": float(rmse_overall),
    }
    return summary, fold_df, oof_df

# ---------------- Run 3 models ----------------
ALL_SUMMARIES, ALL_FOLDS, ALL_OOF = [], [], []

for runner in [run_plsr_nested, run_svr_nested_samplelevel, run_rf_nested]:
    summary, fold_df, oof_df = runner()
    ALL_SUMMARIES.append(summary)
    ALL_FOLDS.append(fold_df)
    ALL_OOF.append(oof_df)

summary_df = pd.DataFrame(ALL_SUMMARIES).sort_values(by="R2_overall_oof", ascending=False).reset_index(drop=True)
folds_df   = pd.concat(ALL_FOLDS, ignore_index=True)
oof_df_all = pd.concat(ALL_OOF, ignore_index=True)

summary_df.to_csv(OUT_DIR / "cv_model_comparison.csv", index=False)
folds_df.to_csv(OUT_DIR / "cv_fold_metrics_3models.csv", index=False)
oof_df_all.to_csv(OUT_DIR / "cv_oof_samples_3models.csv", index=False)

print("\n===== Model comparison (sorted by overall OOF R2) =====")
print(summary_df)

# ---------------- Best model plots (only) ----------------
best_model = summary_df.loc[0, "model"]
best_oof = oof_df_all.loc[oof_df_all["model"] == best_model].copy()
best_oof.to_csv(OUT_DIR / "best_model_oof_samples.csv", index=False)

y_true = best_oof["y_true"].to_numpy(dtype=float)
y_pred = best_oof["y_pred"].to_numpy(dtype=float)
resid  = y_pred - y_true

r2_best = r2_score(y_true, y_pred)
rmse_best = rmse(y_true, y_pred)

# parity
plt.figure(figsize=(6, 6))
plt.scatter(y_true, y_pred, s=28, alpha=0.8)
mn, mx = float(min(y_true.min(), y_pred.min())), float(max(y_true.max(), y_pred.max()))
plt.plot([mn, mx], [mn, mx], lw=2)
plt.xlabel("True protein content (%)")
plt.ylabel("Predicted protein content (%)")
plt.title(f"{best_model} | OOF (sample-level)\nR²={r2_best:.3f}  RMSE={rmse_best:.3f}")
plt.tight_layout()
plt.savefig(OUT_DIR / f"parity_best_{best_model}.png", dpi=200)
plt.close()

# residuals vs true
plt.figure(figsize=(6, 4))
plt.scatter(y_true, resid, s=28, alpha=0.8)
plt.axhline(0.0, lw=2)
plt.xlabel("True protein content (%)")
plt.ylabel("Residual (pred - true)")
plt.title(f"Residuals vs True | {best_model} (OOF)")
plt.tight_layout()
plt.savefig(OUT_DIR / f"residuals_vs_true_best_{best_model}.png", dpi=200)
plt.close()

# residual histogram
plt.figure(figsize=(6, 4))
plt.hist(resid, bins=16, edgecolor="black", alpha=0.9)
plt.xlabel("Residual (pred - true)")
plt.ylabel("Count")
plt.title(f"Residual histogram | {best_model} (OOF)")
plt.tight_layout()
plt.savefig(OUT_DIR / f"residuals_hist_best_{best_model}.png", dpi=200)
plt.close()

print(f"\n[SAVE] All outputs -> {OUT_DIR}")
print(f"[BEST] {best_model} | OOF R²={r2_best:.4f} RMSE={rmse_best:.4f}")


[INFO] patches=14546 | samples=113 | bands=272
[INFO] protein column = protein content
[INFO] SVR uses sample-level data: X=(113, 272), y=(113,)
[PLSR | Fold 1] R2=0.8976 RMSE=0.3584 | best={'n_components': 13}
[PLSR | Fold 2] R2=0.9761 RMSE=0.2411 | best={'n_components': 20}
[PLSR | Fold 3] R2=0.9317 RMSE=0.3590 | best={'n_components': 19}
[PLSR | Fold 4] R2=0.9545 RMSE=0.3630 | best={'n_components': 18}
[PLSR | Fold 5] R2=0.9414 RMSE=0.3151 | best={'n_components': 16}
[SVR_RBF | Fold 1] R2=0.6790 RMSE=0.7346 | best={'C': 10, 'gamma': 'scale', 'epsilon': 0.05}
[SVR_RBF | Fold 2] R2=0.8133 RMSE=0.5536 | best={'C': 10, 'gamma': 'scale', 'epsilon': 0.05}
[SVR_RBF | Fold 3] R2=0.7395 RMSE=0.8587 | best={'C': 10, 'gamma': 'scale', 'epsilon': 0.05}
[SVR_RBF | Fold 4] R2=0.8229 RMSE=0.5743 | best={'C': 10, 'gamma': 'scale', 'epsilon': 0.05}
[SVR_RBF | Fold 5] R2=0.7860 RMSE=0.6564 | best={'C': 10, 'gamma': 'scale', 'epsilon': 0.05}
[RF | Fold 1] R2=0.8493 RMSE=0.4346 | best={'n_estimators': 